In [99]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 80% !important; }</style>"))
pd.set_option("display.max_columns", 65)

In [100]:
from IPython.display import Image

In [101]:
df = pd.read_csv('./datas/final_1.csv')

In [102]:
df1 = df[df.시즌 == 2017]
df2 = df[df.시즌 <= 2016]

In [103]:
def column_new(df):
    # 시즌 카테고리
    df['시즌C'] = df.시즌.apply(lambda x: 0 if x <= 2013 else x)
    # 2014년도에 처음으로 억대 연봉선수 생김
    df['시즌C'] = df.시즌C.apply(lambda x: 1 if 2014 == x else x)
    df['시즌C'] = df.시즌C.apply(lambda x: 2 if x > 2014 else x)

    # 경력
    df['경력'] = df['시즌'] - df['데뷔']

    # 이적횟수
    df['이적'] = None
    for x in df['이름']:
        df['이적'][df['이름'] == x] = len(df[df['이름'] == x]['팀'].unique()) - 1
    df.이적 = df.이적.astype(int)

    # 골든 글러브 카테고리
    df.GG1 = df.GG1.apply(lambda x: 4 if x >= 4 else x)

    # 나이 카테고리
    df['나이C'] = df.나이.apply(lambda x: 0 if x <= 23 else x)
    df['나이C'] = df.나이C.apply(lambda x: 1 if 26 >= x > 23 else x)
    df['나이C'] = df.나이C.apply(lambda x: 2 if 30 >= x > 26 else x)
    df['나이C'] = df.나이C.apply(lambda x: 3 if 35 >= x > 30 else x)
    df['나이C'] = df.나이C.apply(lambda x: 4 if 38 >= x > 35 else x)
    df['나이C'] = df.나이C.apply(lambda x: 5 if x >= 39 else x)

    # 출전횟수
    df['횟수'] = 1
    for id_x in df.이름.unique():
        df["횟수"].loc[df[df["이름"] == id_x].reset_index()["index"]] = list(
            df[df["이름"] == id_x].reset_index().reset_index()["level_0"] + 1)
    # 로그연봉
    df['로그연봉'] = np.log(df['연봉'])

In [104]:
column_new(df1)

In [105]:
column_new(df2)

![](./picture/야구.png)

# 1. 주제

## KBO프로야구 타자 연봉 예측

- KBO 관중수 800만명 이상
- KBO 선수 몸값 거품 문제 대두
- 어떤 기준으로 연봉을 책정하는지에 대해 의문이 생김

# 2. 분석 계획
![](./picture/분석계획.jpeg)

# 3. EDA

![](./picture/시즌기록.png)

![](./picture/연봉.png)

### train/test 데이터 분리
- train(df2) : 시즌 2010 ~ 2017
- test(df1) :  시즌 2018

In [110]:
def season_category():
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go

    fig = make_subplots(
        rows=1, cols=2, shared_xaxes=True, vertical_spacing=0.02,
    )

    a = df2.연봉.groupby([df2.시즌]).mean()
    b = df2.OPS.groupby([df2.시즌]).mean()

    fig.add_trace(go.Bar(x=a.index, y=a, name="시즌별 연봉"), row=1, col=1)
    fig.add_trace(go.Bar(x=b.index, y=b, name="시즌별 OPS"), row=1, col=2)
    fig.update_layout(title_text="시즌별 연봉과 OPS 비교")
    fig.show()

### 시즌 카테고리

In [111]:
season_category()

### 나이 카테고리

In [112]:
def age_category():
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go

    fig = make_subplots(shared_xaxes=True, vertical_spacing=0.02,
    )

    a = df2.연봉.groupby([df2.나이]).mean()
    b = df2.OPS.groupby([df2.나이]).mean()

    fig.add_trace(go.Line(x=a.index, y=a, name="나이별 연봉"), row=1, col=1)
    #fig.add_trace(go.Line(x=b.index, y=b, name="나이별 OPS"), row=1, col=2)

    fig.update_layout(title_text="나이별 연봉과 OPS 비교")
    fig.show()

In [113]:
age_category()

/opt/anaconda3/lib/python3.7/site-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




### 시즌 참가 횟수

In [114]:
def num_category():
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go

    fig = make_subplots(
        rows=1, cols=2, vertical_spacing=0.02)
    test = df2[df2.타석 >= 30]
    a = test.연봉.groupby([test.횟수]).mean()
    b = test.OPS.groupby([test.횟수]).mean()

    test['OPS_round'] = round(test.OPS, 1)
    test1 = test[(test['OPS_round'] >= 0.7) & (test['OPS_round'] <= 0.9)]

    fig.add_trace(go.Bar(x=a.index, y=a, name="시즌 참가 횟수별 연봉"), row=1, col=1)
    fig.add_trace(go.Bar(x=b.index, y=b, name="시즌 참가 횟수별 OPS"), row=1, col=2)
    fig.update_layout(title_text="시즌 참가 횟수별 연봉과 OPS")
    fig.show()

In [115]:
num_category()

### 변수에 스케일링을 한 이유

In [116]:
# 변수마다 차이가 커서 scaling을 해줘야 함
def scaling():
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go
    fig = go.Figure()
    fig.add_trace(go.Box(y=df2['G'], name='G'))
    fig.add_trace(go.Box(y=df2['나이'], name='나이'))
    fig.add_trace(go.Box(y=df2['루타'], name='루타'))
    fig.add_trace(go.Box(y=df2['trans'], name='물가'))
    fig.add_trace(go.Box(y=df2['경력'], name='경력'))
    fig.update_layout(title_text="스케일링 해주는 이유")
    fig.show()

In [117]:
scaling()

### 용병 여부

In [118]:
def for_kor_category():
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go

    kor = df2[df2['용병'] == 0]
    a = kor.연봉.groupby([kor.시즌]).mean()
    fore = df2[df2['용병'] == 1]
    b = fore.연봉.groupby([fore.시즌]).mean()

    fig = go.Figure()

    fig.add_trace(go.Bar(x=a.index, y=a, name="시즌별 한국인  평균 연봉"))
    fig.add_trace(go.Bar(x=b.index, y=b, name="시즌별 용병 평균 연봉"))
    # fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
    # marker_line_width=1.5, opacity=0.6)
    fig.update_layout(title='용병 여부에 따른 평균 연봉')
    fig.show()


In [119]:
for_kor_category()

### 필요한 열 추가

In [120]:
def column_new(df):
    # 시즌 카테고리
    df['시즌C'] = df.시즌.apply(lambda x: 0 if x <= 2013 else x)
    # 2014년도에 처음으로 억대 연봉선수 생김
    df['시즌C'] = df.시즌C.apply(lambda x: 1 if 2014 == x else x)
    df['시즌C'] = df.시즌C.apply(lambda x: 2 if x > 2014 else x)

    # 경력
    df['경력'] = df['시즌'] - df['데뷔']

    # 이적횟수
    df['이적'] = None
    for x in df['이름']:
        df['이적'][df['이름'] == x] = len(df[df['이름'] == x]['팀'].unique()) - 1
    df.이적 = df.이적.astype(int)

    # 골든 글러브 카테고리
    df.GG1 = df.GG1.apply(lambda x: 4 if x >= 4 else x)

    # 나이 카테고리
    df['나이C'] = df.나이.apply(lambda x: 0 if x <= 23 else x)
    df['나이C'] = df.나이C.apply(lambda x: 1 if 26 >= x > 23 else x)
    df['나이C'] = df.나이C.apply(lambda x: 2 if 30 >= x > 26 else x)
    df['나이C'] = df.나이C.apply(lambda x: 3 if 35 >= x > 30 else x)
    df['나이C'] = df.나이C.apply(lambda x: 4 if 38 >= x > 35 else x)
    df['나이C'] = df.나이C.apply(lambda x: 5 if x >= 39 else x)

    # 출전횟수
    df['횟수'] = 1
    for id_x in df.이름.unique():
        df["횟수"].loc[df[df["이름"] == id_x].reset_index()["index"]] = list(
            df[df["이름"] == id_x].reset_index().reset_index()["level_0"] + 1)
    # 로그연봉
    df['로그연봉'] = np.log(df['연봉'])

In [121]:
column_new(df1)
column_new(df2)

# 4. 예측 모델

## 1. 모델 1

In [122]:
formula1 = '연봉 ~ C(포지션) + C(시즌C)+ C(나이C):scale(경력) + scale(G) + scale(루타) + scale(도루) +scale(고4) + scale(타3) + C(용병) +scale(이적) + scale(ISOD) + scale(BBK) +scale(횟수)'
model1 = sm.OLS.from_formula(formula1, data=df2)
result1 = model1.fit()
print(result1.summary())

                            OLS Regression Results                            
Dep. Variable:                     연봉   R-squared:                       0.665
Model:                            OLS   Adj. R-squared:                  0.654
Method:                 Least Squares   F-statistic:                     62.11
Date:                Fri, 17 Apr 2020   Prob (F-statistic):          9.48e-167
Time:                        20:17:43   Log-Likelihood:                -8671.6
No. Observations:                 808   AIC:                         1.740e+04
Df Residuals:                     782   BIC:                         1.752e+04
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept              1.124e+

In [123]:
from sklearn.model_selection import KFold
scores1 = np.zeros(5)
cv = KFold(5, shuffle=True, random_state=0)
for i, (idx_train, idx_test) in enumerate(cv.split(df2)):
    df_train = df2.iloc[idx_train]
    df_test = df2.iloc[idx_test]
    model1 = sm.OLS.from_formula(formula1, data=df_train)
    result1 = model1.fit()
    pred1 = result1.predict(df_test)
    rss = ((df_test.연봉 - pred1) ** 2).sum()
    tss = ((df_test.연봉 - df_test.연봉.mean()) ** 2).sum()
    rsquared = 1 - rss / tss
    scores1[i] = rsquared
    print("학습 R2 = {:.8f}, 검증 R2 = {:.8f}".format(result1.rsquared, rsquared))

학습 R2 = 0.65333763, 검증 R2 = 0.68727395
학습 R2 = 0.69096687, 검증 R2 = 0.46685029
학습 R2 = 0.68058757, 검증 R2 = 0.58916978
학습 R2 = 0.66470085, 검증 R2 = 0.63708486
학습 R2 = 0.66646359, 검증 R2 = 0.43003290


In [124]:
def model1():
    import plotly.graph_objects as go
    import pandas as pd

    pred1 = result1.predict(df2)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df2['연봉'], y=pred1,
                             mode='markers', marker_color=df2['로그연봉']))

    fig.update_layout(title='model 1 : 야구선수 연봉 예측')
    fig.show()


In [125]:
model1()

## 2. 모델 2
- 연봉에 로그를 취함

In [126]:
formula2 = '로그연봉 ~ C(포지션) + C(시즌C)+ C(나이C):scale(경력) + scale(G) + scale(루타) + scale(도루) +scale(고4) + scale(타3) + C(용병) +scale(이적) + scale(ISOD) + scale(BBK) +scale(횟수)'
model2 = sm.OLS.from_formula(formula2, data=df2)
result2 = model2.fit()
print(result2.summary())

                            OLS Regression Results                            
Dep. Variable:                   로그연봉   R-squared:                       0.826
Model:                            OLS   Adj. R-squared:                  0.820
Method:                 Least Squares   F-statistic:                     148.1
Date:                Fri, 17 Apr 2020   Prob (F-statistic):          1.77e-276
Time:                        20:17:45   Log-Likelihood:                -391.84
No. Observations:                 808   AIC:                             835.7
Df Residuals:                     782   BIC:                             957.7
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 9.09

In [158]:
from sklearn.model_selection import KFold

scores2 = np.zeros(5)
cv = KFold(5, shuffle=True, random_state=13)
for i, (idx_train, idx_test) in enumerate(cv.split(df2)):
    df_train = df2.iloc[idx_train]
    df_test = df2.iloc[idx_test]
    model2 = sm.OLS.from_formula(formula2, data=df_train)
    result2 = model2.fit()
    pred2 = result2.predict(df_test)
    rss = ((df_test.로그연봉 - pred2) ** 2).sum()
    tss = ((df_test.로그연봉 - df_test.로그연봉.mean()) ** 2).sum()
    rsquared = 1 - rss / tss
    scores2[i] = rsquared
    print("학습 R2 = {:.8f}, 검증 R2 = {:.8f}".format(result2.rsquared, rsquared))

학습 R2 = 0.82901056, 검증 R2 = 0.80730394
학습 R2 = 0.81776114, 검증 R2 = 0.84552203
학습 R2 = 0.83619361, 검증 R2 = 0.76673304
학습 R2 = 0.81948807, 검증 R2 = 0.83445454
학습 R2 = 0.83049867, 검증 R2 = 0.79348258


In [128]:
def model2():
    import plotly.graph_objects as go
    import pandas as pd

    pred2 = result2.predict(df2)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df2['로그연봉'], y=pred2,
                             mode='markers', marker_color=df2['로그연봉']))
    fig.update_layout(title='야구선수 연봉 예측')
    fig.show()
    


In [129]:
model2()

## 3. 모델 3
 - 높은 연봉을 받는 선수들을 잘 예측하지 못함
     - 수상 지표 추가(크롤링)
     - 팀 순위 추가 (크롤링)

![](./picture/수상.png)

![](./picture/팀순위.png)

In [132]:
df2

,이름,팀,시즌,포지션,생일,G,타석,타수,득점,안타,타1,타2,타3,홈런,루타,타점,도루,도실,볼넷,사구,고4,삼진,병살,희타,희비,타율,출루,장타,OPS,ISOD,BBK,WAR,WAA,계약,참고,MVP,MVP1,수상,ALMVP,ALMVP1,HSMVP,HSMVP1,SPMVP,SPMVP1,GG,GG1,POMVP,POMVP1,신인왕,신인왕1,수상N,수상N1,데뷔,용병,나이,순위,trans,연봉,시즌C,경력,이적,나이C,횟수,로그연봉
0,강민호,롯데,2010.0,C,1985-08-18,117.0,465.0,410.0,66.0,125.0,82.0,19.0,1.0,23.0,215.0,72.0,2.0,7.0,42.0,8.0,1.0,65.0,16.0,0.0,5.0,0.305,0.376,0.524,0.901,0.071,0.049462,5.04,2.325,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2004,0,26.0,4,1.147091,20000,0.0,6.0,0,1.0,1,9.903488
1,장성호,한화,2010.0,1B,1977-10-18,74.0,279.0,237.0,34.0,58.0,46.0,8.0,0.0,4.0,78.0,29.0,0.0,2.0,35.0,2.0,1.0,35.0,9.0,3.0,2.0,0.245,0.344,0.329,0.673,0.099,0.000000,0.06,0.233,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1996,0,34.0,8,1.147091,20000,0.0,14.0,1,3.0,1,9.903488
2,이희근,한화,2010.0,C,1985-06-07,82.0,158.0,136.0,14.0,25.0,24.0,1.0,0.0,0.0,26.0,12.0,2.0,2.0,10.0,6.0,0.0,33.0,6.0,5.0,1.0,0.184,0.268,0.191,0.459,0.084,0.145570,-0.44,0.790,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2008,0,26.0,8,1.147091,3500,0.0,2.0,0,1.0,1,8.160518
3,이대수,한화,2010.0,SS,1981-08-21,125.0,416.0,375.0,33.0,87.0,66.0,13.0,1.0,7.0,123.0,37.0,2.0,4.0,26.0,6.0,0.0,57.0,4.0,6.0,3.0,0.232,0.290,0.328,0.618,0.058,0.074519,0.46,1.733,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2002,0,30.0,8,1.147091,9000,0.0,8.0,1,2.0,1,9.104980
4,오선진,한화,2010.0,3B,1989-07-07,108.0,240.0,221.0,24.0,44.0,39.0,5.0,0.0,0.0,49.0,14.0,7.0,2.0,12.0,2.0,0.0,57.0,7.0,5.0,0.0,0.199,0.247,0.222,0.469,0.048,0.187500,-0.96,0.800,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2008,0,22.0,8,1.147091,3300,0.0,2.0,0,0.0,1,8.101678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
803,김문호,롯데,2016.0,LF,1987-06-22,140.0,599.0,526.0,77.0,171.0,133.0,28.0,3.0,7.0,226.0,70.0,12.0,15.0,62.0,6.0,1.0,70.0,15.0,3.0,2.0,0.325,0.401,0.430,0.831,0.076,0.013356,2.47,-0.643,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2006,0,30.0,8,1.034653,14000,2.0,10.0,0,2.0,6,9.546813
804,정훈,롯데,2016.0,2B,1987-07-18,121.0,439.0,370.0,48.0,97.0,75.0,18.0,2.0,2.0,125.0,46.0,7.0,7.0,43.0,16.0,0.0,75.0,12.0,8.0,2.0,0.262,0.362,0.338,0.700,0.100,0.072893,0.73,-0.043,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2010,0,30.0,8,1.034653,15000,2.0,6.0,0,2.0,7,9.615805
805,김호령,KIA,2016.0,CF,1992-04-30,124.0,514.0,453.0,72.0,121.0,92.0,18.0,3.0,8.0,169.0,41.0,19.0,10.0,36.0,10.0,0.0,104.0,7.0,13.0,2.0,0.267,0.333,0.373,0.706,0.066,0.132296,0.78,1.030,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015,0,25.0,5,1.034653,9500,2.0,1.0,0,1.0,2,9.159047
806,이홍구,KIA,2016.0,C,1990-12-11,106.0,273.0,241.0,27.0,64.0,37.0,16.0,2.0,9.0,111.0,45.0,0.0,1.0,19.0,4.0,1.0,66.0,5.0,3.0,6.0,0.266,0.322,0.461,0.783,0.056,0.172161,1.01,0.051,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2013,0,27.0,5,1.034653,10000,2.0,3.0,0,2.0,3,9.210340


In [133]:
def prize():
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go

    fig = make_subplots(rows=1, cols=2, shared_xaxes=True,
                        vertical_spacing=0.02)

    a = df2.연봉.groupby([df2.수상N1]).mean()
    b = df2.연봉.groupby([df2.수상]).mean()
    

    fig.add_trace(go.Bar(x=a.index, y=a, name="수상 누적 횟수별 연봉"), row=1, col=1)
    fig.add_trace(go.Bar(x=b.index, y=b, name="수상종류별 연봉"), row=1, col=2)

    fig.update_layout(title_text="수상 누적 횟수와 종류별 연봉")
    fig.show()
    


In [134]:
def MVP():
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go  
    fig = go.Figure()
    df2['MVP'] = df2['MVP'].apply(lambda x : '시즌MVP' if x==1 else 0)    
    c = df2.연봉.groupby([df2.MVP]).mean()
    b = df2.연봉.groupby([df2.수상]).mean()
    fig.add_trace(go.Bar(x=b.index, y=b, name="수상종류별 연봉"))
    fig.add_trace(go.Bar(x=c.index, y=c, name="시즌 MVP 수상 여부별 연봉"))
    fig.show()

In [135]:
prize()

In [136]:
MVP()

In [137]:
def team_rank():
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go
    fig = go.Figure()
    temp = df2[df2['팀'] == 'KIA'].groupby(df2['순위']).mean()
    fig.add_trace(go.Bar(x=temp["순위"], y=temp["연봉"], name='KIA'))
    temp1 = df2[df2['팀'] == '삼성'].groupby(df2['순위']).mean()
    fig.add_trace(go.Bar(x=temp1["순위"], y=temp1["연봉"], name='삼성'))
    temp2 = df2[df2['팀'] == '두산'].groupby(df2['순위']).mean()
    fig.add_trace(go.Bar(x=temp2["순위"], y=temp2["연봉"], name='두산'))

    fig.update_layout(title_text="팀당 순위별 연봉 변화")
    fig.show()
    


In [138]:
team_rank()

In [139]:
def position_mvp():
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go

    mvp = df2[df2['MVP1'] == 0]
    mvp1 = df2[df2['MVP1'] == 1]

    a = mvp.연봉.groupby([mvp.포지션]).mean()
    b = mvp1.연봉.groupby([mvp1.포지션]).mean()

    fig = go.Figure()

    fig.add_trace(go.Bar(x=a.index, y=a, name="포지션별 연봉"))
    fig.add_trace(go.Bar(x=b.index, y=b, name="시즌 MVP를 수상 연봉"))

    fig.update_layout(title='포지션별 시즌MVP 수상 여부에 따른 연봉')
    fig.show()
    


In [140]:
position_mvp()

In [141]:
def GG_안타():
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go
    fig = go.Figure()
    ngg = df2[df2.GG1 == 0]

    df2['안타1'] = None
    df2['안타1'] = df2.안타.apply(lambda x: 0 if x <= 17 else x)
    df2['안타1'] = df2.안타1.apply(lambda x: 17 if 56 >= x > 17 else x)
    df2['안타1'] = df2.안타1.apply(lambda x: 56 if 105 >= x > 56 else x)
    df2['안타1'] = df2.안타1.apply(lambda x: 105 if x > 105 else x)

    df2['안타1'] = df2.안타1.apply(lambda x: '0~17' if x == 0 else x)
    df2['안타1'] = df2.안타1.apply(lambda x: '17~56' if x == 17 else x)
    df2['안타1'] = df2.안타1.apply(lambda x: '56~105' if x == 56 else x)
    df2['안타1'] = df2.안타1.apply(lambda x: '105~201' if x == 105 else x)

    temp = df2[df2['GG1'] == 0].groupby(df2['안타1']).mean()
    temp.sort_values('로그연봉', inplace=True)
    fig.add_trace(go.Bar(x=temp.index, y=temp["연봉"], name='GG 수상하지 않은 경우'))
    temp1 = df2[df2['GG1'] >= 1].groupby(df2['안타1']).mean()
    fig.add_trace(go.Bar(x=temp1.index, y=temp1["연봉"], name='GG 수상한 경우'))

    fig.update_layout(title_text="안타와 GG 상호작용 이유")
    fig.show()



In [142]:
GG_안타()

In [143]:
 df2['MVP'] = df2['MVP'].apply(lambda x : 1 if x=='시즌MVP' else 0)   

In [144]:
formula3 = '로그연봉 ~ C(포지션) + C(시즌C)+ C(나이C):scale(경력) + scale(G) + scale(루타) + scale(도루) +scale(고4) + scale(타3) + C(용병) +scale(이적) + scale(ISOD) + scale(BBK) +scale(횟수) + scale(수상N) + scale(수상N1) + scale(WAR)+ scale(ALMVP) +scale(MVP)+ C(팀):scale(순위) + C(GG):scale(안타)'
model3 = sm.OLS.from_formula(formula3, data=df2)
result3 = model3.fit()
print(result3.summary())

                            OLS Regression Results                            
Dep. Variable:                   로그연봉   R-squared:                       0.850
Model:                            OLS   Adj. R-squared:                  0.841
Method:                 Least Squares   F-statistic:                     102.9
Date:                Fri, 17 Apr 2020   Prob (F-statistic):          2.07e-283
Time:                        20:17:53   Log-Likelihood:                -331.84
No. Observations:                 808   AIC:                             749.7
Df Residuals:                     765   BIC:                             951.6
Df Model:                          42                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 9.11

In [145]:
from sklearn.model_selection import KFold
scores3 = np.zeros(5)
cv = KFold(5, shuffle=True, random_state=10)
for i, (idx_train, idx_test) in enumerate(cv.split(df2)):
    df_train = df2.iloc[idx_train]
    df_test = df2.iloc[idx_test]
    model3 = sm.OLS.from_formula(formula3, data=df_train)
    result3 = model3.fit()
    pred3 = result3.predict(df_test)
    rss = ((df_test.로그연봉 - pred3) ** 2).sum()
    tss = ((df_test.로그연봉 - df_test.로그연봉.mean()) ** 2).sum()
    rsquared = 1 - rss / tss
    scores3[i] = rsquared
    print("학습 R2 = {:.8f}, 검증 R2 = {:.8f}".format(result3.rsquared, rsquared))

학습 R2 = 0.85282175, 검증 R2 = 0.82105604
학습 R2 = 0.84325044, 검증 R2 = 0.86767919
학습 R2 = 0.84896284, 검증 R2 = 0.84380050
학습 R2 = 0.85466189, 검증 R2 = 0.81970104
학습 R2 = 0.85809071, 검증 R2 = 0.80734956


In [146]:
def model3():
    import plotly.graph_objects as go
    import pandas as pd
    model3 = sm.OLS.from_formula(formula3, data=df2)
    result3 = model3.fit()
    pred3 = result3.predict(df2)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df2['로그연봉'], y=pred3,
                             mode='markers', marker_color=df2['로그연봉']))
    
    fig.add_trace(go.Scatter(x=[7.5, 10, 12], y=[7.5, 10, 12]))
    fig.update_layout(title='model3 야구선수 연봉 예측')
    fig.show()


In [147]:
def model23():
    import plotly.graph_objects as go
    import pandas as pd
    model3 = sm.OLS.from_formula(formula3, data=df2)
    result3 = model3.fit()
    pred3 = result3.predict(df2)
    fig = go.Figure()

    pred2 = result2.predict(df2)
    fig.add_trace(go.Scatter(x=df2['로그연봉'], y=pred2,
                             mode='markers', name='model2'))
    fig.add_trace(go.Scatter(x=df2['로그연봉'], y=pred3,
                             mode='markers', name='model3'))

    fig.add_trace(go.Scatter(x=[7.5, 10, 12], y=[7.5, 10, 12]))
    fig.update_layout(title='model3 야구선수 연봉 예측')
    fig.show()

In [148]:
model3()

In [149]:
model23()

## 4. 모델 4
 - FA 계약 추가(카테고리)
    - FA 계약 체결 당해년도
    - FA 계약중
    - FA 계약 X
- FA 계약중인 선수는 이전년도의 연봉과 같아서 데이터를 분리한뒤
    - 계약중이 아닌 데이터로 학습 -> 예측
    - 계약중인 데이터는 이전년도의 연봉을 가져옴
    
- 현재가치 반영(trans)

In [150]:
df3 = df2[df2['계약'] == '계약중']

In [151]:
formula4 = '로그연봉 ~ C(포지션) + C(시즌C) + scale(나이):scale(횟수) +scale(경력) + scale(G) + C(GG):scale(안타) + scale(도루) +scale(고4) + scale(타3) + C(용병) + scale(ISOD) +  scale(수상N) + scale(수상N1) + C(계약) + scale(WAR) + scale(이적) + scale(ALMVP) + C(팀):scale(순위) + scale(trans)'
model4 = sm.OLS.from_formula(formula4, data=df2)
result4 = model4.fit()
print(result4.summary())

                            OLS Regression Results                            
Dep. Variable:                   로그연봉   R-squared:                       0.864
Model:                            OLS   Adj. R-squared:                  0.858
Method:                 Least Squares   F-statistic:                     132.4
Date:                Fri, 17 Apr 2020   Prob (F-statistic):          2.06e-305
Time:                        20:17:56   Log-Likelihood:                -290.77
No. Observations:                 808   AIC:                             657.5
Df Residuals:                     770   BIC:                             835.9
Df Model:                          37                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                9.1154 

![](./datas/model4.png)

In [153]:
from sklearn.model_selection import KFold
scores4 = np.zeros(5)
cv = KFold(5, shuffle=True, random_state=5)
for i, (idx_train, idx_test) in enumerate(cv.split(df2)):
    df_train = df2.iloc[idx_train]
    df_test = df2.iloc[idx_test]
    
    df_train = df_train[df_train["계약"] != "계약중"]
    df_test_FA = df_test[df_test["계약"] == "계약중"]['로그연봉']
    df_test_t = df_test[df_test["계약"] != "계약중"]
    
    model4 = sm.OLS.from_formula(formula4, data=df_train)
    result4 = model4.fit()
    pred4 = result4.predict(df_test_t)

    rss = ((df_test.로그연봉 - df_test_FA.append(pred4)) ** 2).sum()
    tss = ((df_test.로그연봉 - df_test.로그연봉.mean()) ** 2).sum()
    rsquared = 1 - rss / tss
    scores4[i] = rsquared
    print("학습 R2 = {:.8f}, 검증 R2 = {:.8f}".format(result4.rsquared, rsquared))

학습 R2 = 0.85140657, 검증 R2 = 0.87876849
학습 R2 = 0.86779841, 검증 R2 = 0.81284255
학습 R2 = 0.86515664, 검증 R2 = 0.82681967
학습 R2 = 0.85093142, 검증 R2 = 0.86826368
학습 R2 = 0.84016935, 검증 R2 = 0.91355677


In [154]:
df3 = df2[df2['계약'] != '계약중']
def model4():
    import plotly.graph_objects as go
    import pandas as pd
    model4 = sm.OLS.from_formula(formula4, data=df3)
    result4 = model4.fit()
    pred4 = result4.predict(df3)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df3['로그연봉'], y=pred4,
                             mode='markers', marker_color=df3['로그연봉']))
    fig.add_trace(go.Scatter(x=[7.5, 10, 12], y=[7.5, 10, 12]))
    fig.update_layout(title='model4 야구선수 연봉 예측')
    fig.show()
    


In [155]:
model4()

In [156]:
def scores():
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go
    scores = [scores1, scores2, scores3, scores4]
    fig = go.Figure()
    fig.add_trace(go.Box(y=scores1, name='model1'))
    fig.add_trace(go.Box(y=scores2, name='model2'))
    fig.add_trace(go.Box(y=scores3, name='model3'))
    fig.add_trace(go.Box(y=scores4, name='model4'))
    
    fig.update_layout(title_text="모델별 성능")
    fig.show()

### 모델별 성능 비교

In [159]:
scores()

# 5. test 데이터로 검증

In [59]:
df1 = df1.reset_index(drop=True)
df1X = df1.drop(['연봉', '로그연봉'], axis=1)
df1y = df1['로그연봉']

df1A = df1[df1.계약 == '계약중']['로그연봉']
df1B = df1X[df1X.계약 != '계약중']
pred = result4.predict(df1B)


In [60]:
df1 = df1.reset_index(drop=True)
df1X = df1.drop(['연봉', '로그연봉'], axis=1)
df1y = df1['로그연봉']

df1A = df1[df1.계약 == '계약중']['로그연봉']
df1B = df1X[df1X.계약 != '계약중']
pred = result4.predict(df1B)

yhat = pd.concat([pred, df1A]).sort_index()

In [61]:
mse = sk.metrics.mean_squared_error(df1y, yhat, sample_weight=None, multioutput='uniform_average')
mae = sk.metrics.mean_absolute_error(df1y, yhat, sample_weight=None, multioutput='uniform_average')
r2 = sk.metrics.r2_score(df1y, yhat, sample_weight=None, multioutput='uniform_average')
mse, mae, r2

(0.0972943924639365, 0.22783022307588816, 0.9055294300200384)

In [62]:
def test():
    import plotly.graph_objects as go

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df1['로그연봉'], y=yhat, mode='markers', marker_color=df1['로그연봉']))
    fig.add_trace(go.Scatter(x=[7.5, 10, 12], y=[7.5, 10, 12]))
    fig.update_layout(title='test 야구선수 연봉 예측 결과')
    fig.show()

In [98]:
test()

## Lasso
- 가중치가 작고, 성능의 변화가 크게 없으므로 정규화를 하지 않아도 됨

In [ ]:
from sklearn.model_selection import validation_curve
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

alphas = np.logspace(-10, -2, 30)
# model.fit_regularized(alpha=0.01, L1_wt=0)
train_scores = []
test_scores = []
for alpha in alphas:
    test_ls = []
    train_ls = []
    #test_score = np.mean(result, X, y, scoring='neg_mean_squared_error', cv=5)
    cv = KFold(5, shuffle=True, random_state=2)
    for i, (idx_train, idx_test) in enumerate(cv.split(df3)):
        df_train = df3.iloc[idx_train]
        df_test = df3.iloc[idx_test]

        model = sm.OLS.from_formula(formula4, data=df_train)
        result = model.fit_regularized(alpha=alpha, L1_wt=1)
        pred = result.predict(df_train)
        train_ls.append(-mean_squared_error(df_train.로그연봉, pred))

        pred = result.predict(df_test)
        test_ls.append(-mean_squared_error(df_test.로그연봉, pred))
    train_scores.append(np.mean(train_ls))
    test_scores.append(np.mean(test_ls))
    print(alpha)

In [ ]:
def lasso():
    optimal_alpha = alphas[np.argmax(test_scores)]
    optimal_score = np.max(test_scores)

    plt.semilogx(alphas, test_scores, '-', label="검증성능")
    plt.semilogx(alphas, train_scores, '--', label="학습성능"),
    plt.axhline(optimal_score, linestyle=':')
    plt.axvline(optimal_alpha, linestyle=':')
    plt.scatter(optimal_alpha, optimal_score)
    plt.ylabel('성능')
    plt.xlabel('정규화 가중치')
    plt.legend()
    plt.show()

In [ ]:
lasso()